# What's in this exercise?
1) Create Storage Intergration
2) Load reference data in staging directory into Snowflake Tables directly and get the performance of Snowflake's elastic performance and scalability.
3) Create FILE FORMAT for the external files
4) Create external stage for ADLS Gen-2 storage account
5) Load csv files available in ADLS Gen-2 Storage into Snowflake tables

In [0]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark.types import StructType, StructField
import os

In [0]:
User = dbutils.secrets.get("snowparkdetails", "username")
Password = dbutils.secrets.get("snowparkdetails", "password")
Account = dbutils.secrets.get("snowparkdetails", "account")
TenandId = dbutils.secrets.get("gen2-storage", "tenant-id")
SchemaName = "taxi"
DatabaseName = "NYCTAXI"
Warehouse = "cluster1"
DBrole = "ACCOUNTADMIN"
CONNECTION_PARAMETERS = {
    'account': Account,
    'user': User,
    'password': Password,
    'schema': SchemaName,
    'database': DatabaseName,
    'warehouse': Warehouse,
    'role':DBrole,
}

session = Session.builder.configs(CONNECTION_PARAMETERS).create()


In [0]:

print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

[Row(CURRENT_WAREHOUSE()='CLUSTER1', CURRENT_DATABASE()='NYCTAXI', CURRENT_SCHEMA()='TAXI')]

In [0]:
# Define source and destination directories
allowed_location = ('azure://demostgacct.blob.core.windows.net/staging','azure://demostgacct.blob.core.windows.net/curated')

session.sql(f"CREATE OR REPLACE STORAGE INTEGRATION azure_integration_gen2 TYPE = EXTERNAL_STAGE \
  STORAGE_PROVIDER = AZURE \
  ENABLED = TRUE \
  AZURE_TENANT_ID = '{TenandId}' \
  STORAGE_ALLOWED_LOCATIONS = {allowed_location}").collect()

Out[4]: [Row(status='Integration AZURE_INTEGRATION_GEN2 successfully created.')]

In [0]:
session.sql("desc storage integration azure_integration").show(max_width=1000)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"property" |"property_type" |"property_value" |"property_default" |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
ENABLED |Boolean |true |false |
STORAGE_PROVIDER |String |AZURE | |
STORAGE_ALLOWED_LOCATIONS |List |azure://[REDACTED]demostgacct.blob.core.windows.net/staging,azure://[REDACTED]demostgacct.blob.core.windows.net/curated |[] |
STORAGE_BLOCKED_LOCATIONS |List | |[] |
AZURE_TENANT_ID |String |[REDACTED] | |
AZURE_CONSENT_URL |String |https://login.microsoftonline.com/[REDACTED]/oauth2/authorize?client_id=de875701-caf5-4d10-aea5-33ca946f3595&response_type=code | |
AZURE_MULTI_TENANT_APP_NAME |String |SnowflakePACInt0662_1647952659280 | |
COMMENT |String | | |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
session.sql(f"CREATE OR REPLACE FILE FORMAT referencedata_csv_format \
            TYPE = CSV \
            COMPRESSION = NONE \
            FIELD_DELIMITER=',' \
            FILE_EXTENSION = 'csv' \
            ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE \
             skip_header=1 ").collect();

In [0]:
session.sql(f"show file formats").show(max_width=50)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"created_on" |"name" |"database_name" |"schema_name" |"type" |"owner" |"comment" |"format_options" |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
2022-05-24 04:58:01.496000-07:00 |MY_JSON_FORMAT |NYCTAXI |TAXI |JSON |ACCOUNTADMIN | |{"TYPE":"json","FILE_EXTENSION":null,"DATE_FORM... |
2022-06-09 03:47:50.795000-07:00 |REFERENCEDATA_CSV_FORMAT |NYCTAXI |TAXI |CSV |ACCOUNTADMIN | |{"TYPE":"CSV","RECORD_DELIMITER":"\n","FIELD_DE... |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
session.sql(f"create or replace stage azure_csv_stage \
  storage_integration = azure_integration \
  url = 'azure://demostgacct.blob.core.windows.net/staging/' \
  file_format = referencedata_csv_format").collect()
  

Out[19]: [Row(status='Stage area AZURE_CSV_STAGE successfully created.')]

In [0]:
session.sql("select * from information_schema.stages").toPandas()

Out[20]:

,STAGE_CATALOG,STAGE_SCHEMA,STAGE_NAME,STAGE_URL,STAGE_REGION,STAGE_TYPE,STAGE_OWNER,COMMENT,CREATED,LAST_ALTERED
0,NYCTAXI,TAXI,AZURE_CSV_STAGE,azure://[REDACTED]demostgacct.blob.core.windows.net...,westus2,External Named,ACCOUNTADMIN,None,2022-06-09 03:48:08.342000-07:00,2022-06-09 03:48:08.406000-07:00


In [0]:
session.sql("list @azure_csv_stage/customer/csvFiles/").show(max_width=1000)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"name" |"size" |"md5" |"last_modified" |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00000-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-108-1-c000.csv |2423786 |4532b55261a23f64df4ab6c62eb40895 |Fri, 27 May 2022 08:48:15 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00001-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-109-1-c000.csv |2423268 |48786b81aadb83782db0b060917e783e |Fri, 27 May 2022 08:48:16 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00002-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-110-1-c000.csv |2427801 |a34cebd103a224413f021cc837c430cd |Fri, 27 May 2022 08:48:15 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00003-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-111-1-c000.csv |2419248 |5870449c4aba96cb5ac1be58f2473dce |Fri, 27 May 2022 08:48:19 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00004-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-112-1-c000.csv |2425732 |bb4543913b6d0440c4e86787abd7cbfd |Fri, 27 May 2022 08:48:21 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00005-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-113-1-c000.csv |2419015 |458109f9c44dc6a66e7314497a61047d |Fri, 27 May 2022 08:48:20 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00006-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-114-1-c000.csv |2422780 |eb27a0f1d757a4c9fc95a401806941af |Fri, 27 May 2022 08:48:17 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00007-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-115-1-c000.csv |2421433 |cd00b545e19f8e13a7261e3773e81575 |Fri, 27 May 2022 08:48:19 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00008-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-116-1-c000.csv |2421195 |672ac836176a52223f38db9473b30088 |Fri, 27 May 2022 08:48:25 GMT |
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/customer/csvFiles/part-00009-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-117-1-c000.csv |2422384 |c3b18626b798eab0e955c591ad8f9c4e |Fri, 27 May 2022 08:48:19 GMT |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
df_stages=session.sql(" list @azure_csv_stage").collect()
for val in df_stages:
  if 'reference-data' in val.name:
    print(val.name)


azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/payment_type_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/rate_code_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/taxi_zone_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/trip_month_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/trip_type_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/vendor_lookup.csv

### List reference datasets

In [0]:
#display(dbutils.fs.ls(srcDataDirRoot)) use below code to list ref data to load
df_stages=session.sql(" list @azure_csv_stage").collect()
for val in df_stages:
  if 'reference-data' in val.name and '.csv' in val.name:
    print(val.name)
    
    
   # @azure_csv_stage/reference-data/taxi_zone_lookup.csv

azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/payment_type_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/rate_code_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/taxi_zone_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/trip_month_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/trip_type_lookup.csv
azure://[REDACTED]demostgacct.blob.core.windows.net/staging/reference-data/vendor_lookup.csv

### 3. Define schema for raw reference data

In [0]:
# 1.  Taxi zone lookup
taxiZoneSchema = StructType([
    StructField("location_id", T.StringType(), True),
    StructField("borough", T.StringType(), True),
    StructField("zone", T.StringType(), True),
    StructField("service_zone", T.StringType(), True)])

#2. Months of the year
tripMonthNameSchema = StructType([
    StructField("trip_month", T.StringType(), True),
    StructField("month_name_short", T.StringType(), True),
    StructField("month_name_full", T.StringType(), True)])

#3.  Rate code id lookup
rateCodeSchema = StructType([
    StructField("rate_code_id", T.IntegerType(), True),
    StructField("description", T.StringType(), True)])

#4.  Payment type lookup
paymentTypeSchema = StructType([
    StructField("payment_type", T.IntegerType(), True),
    StructField("abbreviation", T.StringType(), True),
    StructField("description", T.StringType(), True)])

#5. Trip type
tripTypeSchema = StructType([
    StructField("trip_type", T.IntegerType(), True),
    StructField("description", T.StringType(), True)])


#6. Vendor ID
vendorSchema = StructType([
    StructField("vendor_id", T.IntegerType(), True),
    StructField("abbreviation", T.StringType(), True),
    StructField("description", T.StringType(), True)])

### Load reference data

##### Create function to load data

In [0]:
#def loadReferenceData(srcDatasetName, srcDataFile, destDataDir, srcSchema, delimiter ):
def loadReferenceData(srcDatasetName, srcDataFile, destTableName, srcSchema, delimiter ):
  print("Dataset:  " + srcDatasetName)
  print(".......................................................")
  

  #---Snowpark code
  refDF = session.read \
                    .schema(srcSchema) \
                    .options({"skip_header": 1, "field_delimiter":delimiter ,"FIELD_OPTIONALLY_ENCLOSED_BY": '"' })\
                    .csv(srcDataFile)
      
  print(f"....reading source {srcDataFile} and saving as Snowflake Table {destTableName}")
  refDF.write.mode("overwrite").saveAsTable(destTableName)
  print("....done")


##### Loading data

In [0]:
srcDataDirRoot="@azure_csv_stage/reference-data/"

loadReferenceData("taxi zone",srcDataDirRoot + "taxi_zone_lookup.csv","taxi_zone_lookup",taxiZoneSchema,",")
loadReferenceData("trip month",srcDataDirRoot + "trip_month_lookup.csv","trip_month_lookup",tripMonthNameSchema,",")
loadReferenceData("rate code",srcDataDirRoot + "rate_code_lookup.csv","rate_code_lookup",rateCodeSchema,",")
loadReferenceData("payment type",srcDataDirRoot + "payment_type_lookup.csv","payment_type_lookup",paymentTypeSchema,",")
loadReferenceData("trip type",srcDataDirRoot + "trip_type_lookup.csv","trip_type_lookup",tripTypeSchema,",")
loadReferenceData("vendor",srcDataDirRoot + "vendor_lookup.csv","vendor_lookup",vendorSchema,",")

In [0]:
session.sql("select * from trip_type_lookup").toPandas()

Out[13]:

,TRIP_TYPE,DESCRIPTION
0,1,Street-hail
1,2,Dispatch


In [0]:
session.sql("select * from taxi_zone_lookup").limit(10).toPandas()

Out[11]:

,LOCATION_ID,BOROUGH,ZONE,SERVICE_ZONE
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


##### Validate load

In [0]:
session.sql("select * from taxi_zone_lookup").toPandas()
session.sql("select * from trip_month_lookup").toPandas()
session.sql("select * from rate_code_lookup").toPandas()
session.sql("select * from payment_type_lookup").toPandas()
session.sql("select * from trip_type_lookup").toPandas()
session.sql("select * from vendor_lookup").toPandas()

Out[12]:

,VENDOR_ID,ABBREVIATION,DESCRIPTION
0,1,Creative Mobile Technologies LLC,Creative Mobile Technologies LLC
1,2,VeriFone Inc.,VeriFone Inc.
